# Bài toán "Start Date to End Date"

## Vấn đề của bài toán

Cần phải tính toán tại **1 thời điểm cụ thể** *(năm, tháng, ngày, giờ,... nào đó)* có bao nhiêu người/vật có mặt.

VD: 
- Vào ngày xx/xx/xxxx có bao nhiêu sản phẩm đang lưu kho (lưu ý mỗi sản phẩm có thời gian lưu kho và xuất kho khác nhau)?
- Vào ngày xx/xx/xxxx có bao nhiêu nhân viên đang được chỉ định vào làm việc cho một dự án nào đó?

Tuy nhiên, khoảng thời gian từ **ngày bắt đầu** đến **ngày kết thúc** của mỗi người/vật là khác nhau.

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Get data path & data name
data_path = 'C:\\Users\\duyph\\Documents\\DAP2308R1\\data\\'
data_name = 'erd_demo.xlsx'

In [3]:
allocated_sheet_name = 'allocated'
employee_sheet_name = 'emp_info'
customer_sheet_name = 'cus_info'

In [4]:
# Load allocated data to Pandas object
allc = pd.read_excel(data_path + data_name
                     ,sheet_name = allocated_sheet_name)
allc.head()

,emp_id,allocated_from,allocated_to,customer_code
0,ductt,2023-02-15,2023-04-30,git
1,manhdt1,2023-02-01,2023-07-04,sld
2,khoand26,2023-04-12,2023-04-30,sld
3,lanntt,2023-03-15,2023-06-15,git
4,anhntt21,2023-07-01,2023-09-30,bii


Như ví dụ này, mỗi nhân viên được chỉ định làm việc cho một khách hàng cụ thể từ ngày bao nhiêu tới ngày bao nhiêu.

Do đó trong cùng 1 ngày, có các trường hợp sau đây:
1. Không có nhân viên nào đang được chỉ định
2. Có 1 nhân viên đang được chỉ định làm việc cho 1 khách hàng cụ thể
3. Có nhiều nhân viên đang được chỉ định làm việc cho nhiều khách hàng khác nhau

Trường hợp 2 và 3 được gom lại thành 1 nhóm (1 hoặc nhiều). Và chúng ta chỉ đang có data của nhóm này thôi.
Do đó, việc cần làm lúc này là tạo ra 1 list các ngày từ đầu năm đến cuối năm, và trong đó phân biệt rõ ngày nào ở trường hợp 1, ngày nào ở trường hợp 2 và 3.

In [5]:
not_allc = allc.loc[:,['emp_id']]
not_allc.head()

,emp_id
0,ductt
1,manhdt1
2,khoand26
3,lanntt
4,anhntt21


In [6]:
not_allc.drop_duplicates(inplace = True)

In [7]:
not_allc['emp_id'].value_counts()

ductt       1
manhdt1     1
khoand26    1
lanntt      1
anhntt21    1
nhansm3     1
longtt      1
lamd6       1
hoattt      1
Name: emp_id, dtype: int64

In [11]:
not_allc['all_2023_date'] = [pd.date_range(start = '1/1/2023', end = '12/31/2023').tolist()\
                             for _ in range(not_allc.shape[0])]
not_allc.head()

,emp_id,all_2023_date
0,ductt,"[2023-01-01 00:00:00, 2023-01-02 00:00:00, 202..."
0,ductt,"[2023-01-01 00:00:00, 2023-01-02 00:00:00, 202..."
0,ductt,"[2023-01-01 00:00:00, 2023-01-02 00:00:00, 202..."
0,ductt,"[2023-01-01 00:00:00, 2023-01-02 00:00:00, 202..."
0,ductt,"[2023-01-01 00:00:00, 2023-01-02 00:00:00, 202..."


In [13]:
not_allc = not_allc.explode('all_2023_date')

In [14]:
not_allc.head()

,emp_id,all_2023_date
0,ductt,2023-01-01
0,ductt,2023-01-02
0,ductt,2023-01-03
0,ductt,2023-01-04
0,ductt,2023-01-05


Kết quả, chúng ta đã có 1 danh sách 365 ngày cho từng nhân viên. Những ngày này đang được mặc định là "chưa có việc".

Việc tiếp theo là chúng ta tìm danh sách những ngày mà 1 nhân viên nào đó "đang có việc". Bằng cách tạo 1 danh sách những ngày từ **Ngày bắt đầu** đến **Ngày kết thúc** công việc.

In [15]:
allc.dtypes

emp_id                    object
allocated_from    datetime64[ns]
allocated_to      datetime64[ns]
customer_code             object
dtype: object

In [16]:
allc['allocated_range'] = [pd.date_range(allc['allocated_from'][_], allc['allocated_to'][_]).tolist()\
                           for _ in range(allc.shape[0])]

In [17]:
allc.head()

,emp_id,allocated_from,allocated_to,customer_code,allocated_range
0,ductt,2023-02-15,2023-04-30,git,"[2023-02-15 00:00:00, 2023-02-16 00:00:00, 202..."
1,manhdt1,2023-02-01,2023-07-04,sld,"[2023-02-01 00:00:00, 2023-02-02 00:00:00, 202..."
2,khoand26,2023-04-12,2023-04-30,sld,"[2023-04-12 00:00:00, 2023-04-13 00:00:00, 202..."
3,lanntt,2023-03-15,2023-06-15,git,"[2023-03-15 00:00:00, 2023-03-16 00:00:00, 202..."
4,anhntt21,2023-07-01,2023-09-30,bii,"[2023-07-01 00:00:00, 2023-07-02 00:00:00, 202..."


In [18]:
allc = allc.explode('allocated_range')

In [19]:
allc

,emp_id,allocated_from,allocated_to,customer_code,allocated_range
0,ductt,2023-02-15,2023-04-30,git,2023-02-15
0,ductt,2023-02-15,2023-04-30,git,2023-02-16
0,ductt,2023-02-15,2023-04-30,git,2023-02-17
0,ductt,2023-02-15,2023-04-30,git,2023-02-18
0,ductt,2023-02-15,2023-04-30,git,2023-02-19
...,...,...,...,...,...
8,hoattt,2023-05-01,2023-06-30,sld,2023-06-26
8,hoattt,2023-05-01,2023-06-30,sld,2023-06-27
8,hoattt,2023-05-01,2023-06-30,sld,2023-06-28
8,hoattt,2023-05-01,2023-06-30,sld,2023-06-29


In [20]:
allc.dtypes

emp_id                     object
allocated_from     datetime64[ns]
allocated_to       datetime64[ns]
customer_code              object
allocated_range    datetime64[ns]
dtype: object

Sau khi hoàn thành danh sách, chúng ta thêm cột mới để đánh dấu trạng thái **Đang có việc (Allocated)** của nhân viên đó.

In [21]:
allc['status'] = 'Allocated'

In [23]:
allc.head()

,emp_id,allocated_from,allocated_to,customer_code,allocated_range,status
0,ductt,2023-02-15,2023-04-30,git,2023-02-15,Allocated
0,ductt,2023-02-15,2023-04-30,git,2023-02-16,Allocated
0,ductt,2023-02-15,2023-04-30,git,2023-02-17,Allocated
0,ductt,2023-02-15,2023-04-30,git,2023-02-18,Allocated
0,ductt,2023-02-15,2023-04-30,git,2023-02-19,Allocated


Cuối cùng, chúng ta **join** danh sách những ngày Đang có việc vào danh sách 365 ngày, sẽ tìm ra được những ngày trống việc.

In [25]:
status_by_day = pd.merge(not_allc, allc
                         , how = 'left'
                         , left_on = ['emp_id', 'all_2023_date']
                         , right_on = ['emp_id', 'allocated_range'])
status_by_day.head()

,emp_id,all_2023_date,allocated_from,allocated_to,customer_code,allocated_range,status
0,ductt,2023-01-01,NaT,NaT,NaN,NaT,NaN
1,ductt,2023-01-02,NaT,NaT,NaN,NaT,NaN
2,ductt,2023-01-03,NaT,NaT,NaN,NaT,NaN
3,ductt,2023-01-04,NaT,NaT,NaN,NaT,NaN
4,ductt,2023-01-05,NaT,NaT,NaN,NaT,NaN


In [26]:
status_by_day['month'] = status_by_day['all_2023_date'].dt.month

In [27]:
status_by_day.groupby(['month', 'status'])['emp_id'].nunique()

month  status   
2      Allocated    2
3      Allocated    4
4      Allocated    7
5      Allocated    4
6      Allocated    4
7      Allocated    3
8      Allocated    2
9      Allocated    2
10     Allocated    1
11     Allocated    1
12     Allocated    1
Name: emp_id, dtype: int64